# wprowadzenie i GIL

## Wątki

pakiet threading - do programowania wielowątkowego


Uruchomienie funkcji w nowym wątku jest proste:

In [1]:
from random import randint
from time import sleep
from threading import Thread

# Thread(group=None, target=None, name=None, args=(), kwargs={})
# group - zarezerwowane na przyszłość
# target - funkcja do wywołania
# name - unikalna nazwa wątku
# args, kwargs - argumenty przekazywane do funkcji


def f(*args, **kwargs):
    print "f({} ,{})".format(args, kwargs)
    sleep(randint(0, 3))
    return


for i in range(5):
    t = Thread(target=f, args=(i,), kwargs={'kwa': 'rg'})
    t.start()  # uruchomienie wątku 
    #t.join()  # oczekiwanie przez wątek wywołujący na wykonanie wątku wywoływanego
    
print "asdf"

f((1,) ,{'kwa': 'rg'})
f((0,) ,{'kwa': 'rg'})
f((2,) ,{'kwa': 'rg'})
f((3,) ,{'kwa': 'rg'})
asdf
f((4,) ,{'kwa': 'rg'})


Wątek można stworzyć dziedzicząc po klasie Thread

In [8]:
from threading import Thread


class NewThread(Thread):
    def __init__(self, *args, **kwargs):
        super(NewThread, self).__init__()
        self.args = args
        self.kwargs = kwargs
        
    def run(self):
        # metoda wywoływana przy wywołaniu .start()
        print 'thread is running'
        print self.args
        print self.kwargs
    
    
t = NewThread(1, 2, 3, 4, 5, a=1, b=3)
t.start()
t.join()

thread is running
(1, 2, 3, 4, 5)
{'a': 1, 'b': 3}


## Współdzielenie zasobów

ograniczenie dostępu do seksji krytycznej przez pechanizm:
* Lock/RLock
* Semafor
* Event

### Lock/RLock

Lock i RLock działają podobnie - jedyna różnica to ta, że RLock może być zwolniony tylko przez wątek, który go zajął

In [21]:
from random import randint
from time import sleep
from threading import Thread, Lock

def print_thread(thread):
    print 'Wykonuje watek {}'.format(thread)
    sleep(randint(0, 2))  # sekcja krytyczna
    print 'Konczy watek {}'.format(thread)
    
    
t1 = Thread(target=print_thread, args=(1,))
t2 = Thread(target=print_thread, args=(2,))
t3 = Thread(target=print_thread, args=(3,))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

print 50 * '='
def print_thread_lock(thread, lock):
    #try:  # eksperyment - zwolanianie locka przez inny wątek - na klasie Lock DZIAŁA
    #    lock.release()
    #except:
    #    pass
    lock.acquire()  # lock, zostaje przechwycony
    print 'Wykonuje watek {}'.format(thread)
    sleep(randint(0, 2))  # sekcja krytyczna
    print 'Konczy watek {}'.format(thread)
    lock.release()  # lock zostaje zwolniony - nie zwolnienie blokady zablokuje inne wątki
    

lock = Lock()    
t1 = Thread(target=print_thread_lock, args=(1, lock))
t2 = Thread(target=print_thread_lock, args=(2, lock))
t3 = Thread(target=print_thread_lock, args=(3, lock))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

print 50 * '=' + '\n\n'
def print_thread_lock_with(thread, lock):
    with lock:  # to samo co wcześniej tylko z managerem kontekstu
        print 'Wykonuje watek {}'.format(thread)
        sleep(randint(0, 2))  # sekcja krytyczna
        print 'Konczy watek {}'.format(thread)
    

lock = Lock()    
t1 = Thread(target=print_thread_lock_with, args=(1, lock))
t2 = Thread(target=print_thread_lock_with, args=(2, lock))
t3 = Thread(target=print_thread_lock_with, args=(3, lock))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

Wykonuje watek 1
Wykonuje watek 2 Konczy watek 1
Wykonuje watek 3

Konczy watek 2Konczy watek 3
Wykonuje watek 1

Konczy watek 1
Wykonuje watek 2
Konczy watek 2Wykonuje watek 3

Konczy watek 3


Wykonuje watek 1
Konczy watek 1Wykonuje watek 2

Konczy watek 2Wykonuje watek 3

Konczy watek 3


In [26]:
# RLock

from random import randint
from time import sleep
from threading import Thread, RLock


def print_thread_lock(thread, lock):
    #try:  # eksperyment - zwolanianie locka przez inny wątek - na RLock NIE DZIAŁA
    #    lock.release()
    #except:
    #    pass
    lock.acquire()  # lock, zostaje przechwycony
    print 'Wykonuje watek {}'.format(thread)
    sleep(randint(0, 2))  # sekcja krytyczna
    print 'Konczy watek {}'.format(thread)
    lock.release()  # lock zostaje zwolniony - nie zwolnienie blokady zablokuje inne wątki
    

lock = RLock()    
t1 = Thread(target=print_thread_lock, args=(1, lock))
t2 = Thread(target=print_thread_lock, args=(2, lock))
t3 = Thread(target=print_thread_lock, args=(3, lock))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

print 50 * '=' + '\n\n'
def print_thread_lock_with(thread, lock):
    with lock:  # to samo co wcześniej tylko z managerem kontekstu
        print 'Wykonuje watek {}'.format(thread)
        sleep(randint(0, 2))  # sekcja krytyczna
        print 'Konczy watek {}'.format(thread)
    

lock = RLock()    
t1 = Thread(target=print_thread_lock_with, args=(1, lock))
t2 = Thread(target=print_thread_lock_with, args=(2, lock))
t3 = Thread(target=print_thread_lock_with, args=(3, lock))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

Wykonuje watek 1
Konczy watek 1
Wykonuje watek 2
Konczy watek 2Wykonuje watek 3==================================================




Wykonuje watek 1Konczy watek 3Wykonuje watek 2


Konczy watek 1Wykonuje watek 3Konczy watek 2


Konczy watek 3


### Semafor

Semafor pozwala kilku wątkom na dostęp do jednego zasobu - zawiera wewnętrzna wartosć określającą ile wątków może mieć dostęp do zasobu.

Zajmowanie semafora:
- jeżeli wartość semafora jest >= 0, wątek obniża wartosć o 1 i kontynuuje wykonanie swojego kodu
